In [103]:
import sys
sys.path.append("C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/MWLrepo/MiWayLife2/Data_Analysis_Package")
from data_preparation import DataPreparation, Analysis, multi_data_ops
import data_operations as do

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

# Loading data

In [2]:
path = "C:/Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/Lapse Rate Projections/Data/"

In [3]:
calls_data = pd.DataFrame(pd.read_csv(path+"call_reason.csv"))
sales_data = pd.DataFrame(pd.read_csv(path+"sales.csv"))
policy_data = pd.DataFrame(pd.read_csv(path+"policy_data.csv"))
payment_data = pd.concat([pd.DataFrame(pd.read_csv(path+"Payment_data_1.csv")),pd.DataFrame(pd.read_csv(path+"Payment_data_2.csv"))])
lapse_data = pd.concat([pd.DataFrame(pd.read_csv(path+"lapse_1.csv")),pd.DataFrame(pd.read_csv(path+"lapse_2.csv"))])


In [4]:
calls_data

,policy_name,call_id,contact_indicator,datetime_start,resolution_code
0,MQ22-0000002,fa3ff83f-39ed-4e47-bf4b-08cc517a3a02,1,2022-08-31 06:07:16,web_quote_declined
1,MQ22-0000002,86f9ce30-dad1-4d94-8593-70eaf788ff78,1,2022-08-31 06:51:27,web_quote_declined
2,ML22-0000003,817c8493-27db-4e6b-8f03-bcbfcd51aa28,0,2022-08-31 10:23:12,completed
3,ML22-0000003,e5af692f-1fcf-48ee-91c6-97ae7d8f9367,0,2022-08-31 10:24:18,completed
4,ML22-0000014,d631ead4-a76e-46a7-8060-83b084a82ddb,1,2022-09-02 07:46:01,sale
...,...,...,...,...,...
949047,MQ25-0412959,a52269fa-1356-49e4-bc0d-0121fa917d66,0,2025-08-06 15:52:09,NaN
949048,SQ25-0432008,3123a2f3-e524-4ac1-b16a-f4cd1aad4197,0,2025-08-06 15:53:03,uw_manual
949049,MQ25-0428537,bc4d39bc-0774-4767-bb2d-c848a5652111,0,2025-08-06 15:53:10,NaN
949050,MQ25-0428537,4524171f-439f-42db-bab6-9281fd681254,0,2025-08-06 17:42:45,NaN


# Filtering data to relevant policies

Relevant policies are policies that have been incepted. The rpt_datamodel_lapse has incepted policies only, hence filtering on policies within that dataset

In [5]:
pol_names = list(lapse_data['policy_name'])
sales_data = sales_data[sales_data['policy_name'].isin(pol_names)]
pol_ids= list(sales_data['policy_id'])
calls_data = calls_data[calls_data['policy_name'].isin(pol_names)]
payment_data = payment_data[payment_data['policy_id'].isin(pol_ids)]
policy_data = policy_data[policy_data['policy_name'].isin(pol_names)]

In [6]:
#Make date column of data type datetime
sales_data=do.dtype_datetime(sales_data)
calls_data=do.dtype_datetime(calls_data)
payment_data=do.dtype_datetime(payment_data)
policy_data=do.dtype_datetime(policy_data)
lapse_data=do.dtype_datetime(lapse_data)


C:\Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/MWLrepo/MiWayLife2/Data_Analysis_Package\data_operations.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_datetime(df[col], errors='coerce')


In [7]:
len(set(pol_names)),len(set(pol_ids)),sales_data['policy_id'].nunique(), calls_data['policy_name'].nunique(), payment_data['policy_id'].nunique(),policy_data['policy_name'].nunique()

(65364, 65364, 65364, 39590, 63981, 65364)

# Aggregating Calla data, Payments data and Lapse data

## Calls data

<ol> <li>Remove call records that are related to cancellation of policy</li>
<li>Categorise the resolution codes</li>
<li>Add the policy's sale date</li>
<li>Filter call records to calls made after the sale date</li>
<li>Aggregate the calls data to policy_name</li>
</ol>

In [8]:
calls_data

,policy_name,call_id,contact_indicator,datetime_start,resolution_code
2,ML22-0000003,817c8493-27db-4e6b-8f03-bcbfcd51aa28,0,2022-08-31 10:23:12,completed
3,ML22-0000003,e5af692f-1fcf-48ee-91c6-97ae7d8f9367,0,2022-08-31 10:24:18,completed
4,ML22-0000014,d631ead4-a76e-46a7-8060-83b084a82ddb,1,2022-09-02 07:46:01,sale
12,ML22-0000005,4d104dc3-ece8-4226-ac34-404d4af38a9f,0,2022-09-02 10:54:29,completed
13,ML22-0000005,2375400c-833e-4cc2-90b1-95baa946c9c7,0,2022-09-02 12:21:04,completed
...,...,...,...,...,...
948982,MWL008540683,6308801b-9229-422d-a4b5-4a73f2f37722,0,2025-08-06 14:55:04,NaN
948988,ML23-0118810,f591438c-c444-425a-80a5-a5d224858351,0,2025-08-06 14:56:34,NaN
948992,ML25-0389266,1a9f33ad-c152-4f35-bb83-86b7c36ea982,0,2025-08-06 14:59:12,NaN
948997,ML23-0056798,99d0c75a-a769-48d9-bd26-3381ed9c05e0,1,2025-08-06 15:06:34,completed


In [9]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# NOTE: calls data might introduct target leakage due to calls being made to policyholders when payments are not made or when cancellation is requested.
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

#Remove calls hade with the following resolution codes
# related to cancellation resolution codes ['retentions_policy_cancelled','policy_cancelled','unpaid_cancelled_policy']

calls = DataPreparation(df=calls_data)
calls.cleaning_ops(drop_nulls = 'N',filtering = 'Y',filter_conditions_exclude={'resolution_code':['retentions_policy_cancelled','policy_cancelled','unpaid_cancelled_policy']})
calls.df.shape
calls_data=calls.df

C:\Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/MWLrepo/MiWayLife2/Data_Analysis_Package\data_operations.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = pd.to_datetime(df[col], errors='coerce')
C:\Users/E1005279/OneDrive - Sanlam Life Insurance Limited/MWL/MWLrepo/MiWayLife2/Data_Analysis_Package\data_preparation.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df.drop_duplicates(inplace=True)


(329630, 5)

In [10]:
# creating buckets for resolution code to hot-one encode it and not have too many columns

def res_code_categories(code):
    if code in ['debit_date_changed', 'cover_reduced','requested_changes_completed','banking_details_changed','alternative_payer','cover_increased','commencement_moved',
                'additional_lives_update','bmi_update','banking_details_update', 'beneficiary_update','occupation_update','personal_details_update',
                 'educational_level_update','income_brackets_update','alcohol_intake_update']:
        return "update"
    elif code in ['claim_no_docs','claim_no_claim','claim_repudiated', 'claim','claim_accepted']:
        return "claims"
    elif code in['qa_no_risk','qa_risk_absorbed',  'qa_recaptured']:
        return "QA"
    elif code in ['sent_to_retentions', 'free_cover','premium_holiday_added','unpaid_policy_reduced', 'second_debit_scheduled','once_off_payment', 'policy_retained', 'unpaid_collect_again',
                  'policy_reinstated', 'policy_disputed','double_debit_scheduled']:
        return "payment"
    elif code in ['completed', 'no_contact_post_sale','product_explanation', 'sum_assured_disclosed', 'confirm_acceptance','converted', ]:
        return "sale"
    elif code in ['duplicate', 'no_changes_requested','unreachable_no_answer', 'testing', 'auto_close', 'sale','sysytem_duplicate', 'cannot_offer_cover_not_the_life_insured', 'no_sale_web_preferred',
       'not_interested_no_reason','not_interested_unreachable_after_initial_contact','not_interested_does_not_need_cover', 'unreachable_bulk_close',
       'unconverted',  'unreachable_voicemail_only','web_quote_declined', 'no_sale_quote_expired',  'not_interested_in_a_will','not_interested_competitors_already_covered', 
       'reading_script','note_on_behalf_of_sales_agent','unreachable_hung_up',  'cannot_offer_cover_age_too_old','no_sale_unreachable_after_initial_contact', 'documents_posted',
       'web_quote_not_interested_after_follow_up', 'not_interested_irate_do_not_call', 'no_sale_price_affordability']:
        return "other"
    else:
        return np.nan
    
calls_data['res_code']= calls_data['resolution_code'].apply(res_code_categories)
calls_data




C:\Users\E1005279\AppData\Local\Temp\ipykernel_28560\2076976377.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calls_data['res_code']= calls_data['resolution_code'].apply(res_code_categories)


,policy_name,call_id,contact_indicator,datetime_start,resolution_code,res_code
2,ML22-0000003,817c8493-27db-4e6b-8f03-bcbfcd51aa28,0,2022-08-31 10:23:12,completed,sale
3,ML22-0000003,e5af692f-1fcf-48ee-91c6-97ae7d8f9367,0,2022-08-31 10:24:18,completed,sale
4,ML22-0000014,d631ead4-a76e-46a7-8060-83b084a82ddb,1,2022-09-02 07:46:01,sale,other
12,ML22-0000005,4d104dc3-ece8-4226-ac34-404d4af38a9f,0,2022-09-02 10:54:29,completed,sale
13,ML22-0000005,2375400c-833e-4cc2-90b1-95baa946c9c7,0,2022-09-02 12:21:04,completed,sale
...,...,...,...,...,...,...
948982,MWL008540683,6308801b-9229-422d-a4b5-4a73f2f37722,0,2025-08-06 14:55:04,NaN,NaN
948988,ML23-0118810,f591438c-c444-425a-80a5-a5d224858351,0,2025-08-06 14:56:34,NaN,NaN
948992,ML25-0389266,1a9f33ad-c152-4f35-bb83-86b7c36ea982,0,2025-08-06 14:59:12,NaN,NaN
948997,ML23-0056798,99d0c75a-a769-48d9-bd26-3381ed9c05e0,1,2025-08-06 15:06:34,completed,sale


In [11]:
#Adding sales date to calls data to filter to calls made after sale
calls_sale_date = multi_data_ops(file_list=[],df1=calls_data, df2=policy_data)
calls_sale_date.col_intersect

{'df1 | df2': {'policy_name'}}

In [12]:
calls_sale_date.merged_df = calls_sale_date.merging(on='policy_name', df_left=calls_sale_date.df1, df_right=calls_sale_date.df2, how='left', col_right=['policy_name','sale_date'])
calls_sale_date.merged_df

,policy_name,call_id,contact_indicator,datetime_start,resolution_code,res_code,sale_date
0,ML22-0000003,817c8493-27db-4e6b-8f03-bcbfcd51aa28,0,2022-08-31 10:23:12,completed,sale,2022-08-31
1,ML22-0000003,e5af692f-1fcf-48ee-91c6-97ae7d8f9367,0,2022-08-31 10:24:18,completed,sale,2022-08-31
2,ML22-0000014,d631ead4-a76e-46a7-8060-83b084a82ddb,1,2022-09-02 07:46:01,sale,other,2022-09-06
3,ML22-0000005,4d104dc3-ece8-4226-ac34-404d4af38a9f,0,2022-09-02 10:54:29,completed,sale,2022-09-02
4,ML22-0000005,2375400c-833e-4cc2-90b1-95baa946c9c7,0,2022-09-02 12:21:04,completed,sale,2022-09-02
...,...,...,...,...,...,...,...
329625,MWL008540683,6308801b-9229-422d-a4b5-4a73f2f37722,0,2025-08-06 14:55:04,NaN,NaN,2022-08-17
329626,ML23-0118810,f591438c-c444-425a-80a5-a5d224858351,0,2025-08-06 14:56:34,NaN,NaN,2023-11-14
329627,ML25-0389266,1a9f33ad-c152-4f35-bb83-86b7c36ea982,0,2025-08-06 14:59:12,NaN,NaN,2025-05-29
329628,ML23-0056798,99d0c75a-a769-48d9-bd26-3381ed9c05e0,1,2025-08-06 15:06:34,completed,sale,2023-08-07


In [13]:
#Selecting only call records after sale date
calls_data = calls_sale_date.merged_df[calls_sale_date.merged_df['datetime_start'].dt.strftime('%Y-%m-%d')>calls_sale_date.merged_df['sale_date'].dt.strftime('%Y-%m-%d')]


In [14]:
calls_data

,policy_name,call_id,contact_indicator,datetime_start,resolution_code,res_code,sale_date
6,ML22-0000005,302c00a7-8968-491d-8572-b7ee66a733af,0,2022-09-05 08:29:38,completed,sale,2022-09-02
270,ML22-0000180,be3c6b14-e50e-4b95-b3ef-9f6fadb40949,0,2022-09-21 14:59:24,completed,sale,2022-09-16
274,ML22-0000244,c069d070-6a63-4f13-933c-fdd36e0a9cbf,0,2022-09-21 15:09:21,completed,sale,2022-09-20
341,ML22-0000098,e1deeacb-d26e-4e11-ab97-a4141ea24a19,0,2022-09-26 07:15:42,completed,sale,2022-09-09
345,ML22-0000098,2ce916e4-6407-4013-85c1-990026858291,1,2022-09-26 08:39:40,completed,sale,2022-09-09
...,...,...,...,...,...,...,...
329625,MWL008540683,6308801b-9229-422d-a4b5-4a73f2f37722,0,2025-08-06 14:55:04,NaN,NaN,2022-08-17
329626,ML23-0118810,f591438c-c444-425a-80a5-a5d224858351,0,2025-08-06 14:56:34,NaN,NaN,2023-11-14
329627,ML25-0389266,1a9f33ad-c152-4f35-bb83-86b7c36ea982,0,2025-08-06 14:59:12,NaN,NaN,2025-05-29
329628,ML23-0056798,99d0c75a-a769-48d9-bd26-3381ed9c05e0,1,2025-08-06 15:06:34,completed,sale,2023-08-07


In [15]:
calls = DataPreparation(df=calls_data)

In [16]:
calls.null_df

,nulls,percentage nulls
policy_name,0,0.000000
call_id,0,0.000000
contact_indicator,0,0.000000
datetime_start,0,0.000000
resolution_code,2337,0.943141
res_code,2337,0.943141
sale_date,0,0.000000


In [17]:
calls.uniq_df

,no. of unique entries
policy_name,34470
call_id,247789
contact_indicator,2
datetime_start,244813
resolution_code,69
res_code,6
sale_date,2870


In [18]:
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# NOTE: calls data might introduct target leakage due to calls being made to policyholders when payments are not made or when cancellation is requested.
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

agg_dict={'# calls': pd.NamedAgg(column='call_id', aggfunc=pd.Series.nunique),
          '# calls contacted': pd.NamedAgg(column='contact_indicator', aggfunc='sum')
          }

calls.aggregation(groupby_cols=['policy_name'],cat_cols=['res_code'] ,agg_dict_not_cat_cols=agg_dict)
calls_data = calls.agg_df
calls.agg_df

,policy_name,# calls,# calls contacted,res_code_QA_sum,res_code_claims_sum,res_code_other_sum,res_code_payment_sum,res_code_sale_sum,res_code_update_sum
0,ML22-0000001,2,1,0,0,0,0,2,0
1,ML22-0000005,13,1,0,0,9,0,4,0
2,ML22-0000014,2,1,0,0,0,0,2,0
3,ML22-0000016,12,0,0,0,0,2,10,0
4,ML22-0000031,24,17,0,0,0,4,20,0
...,...,...,...,...,...,...,...,...,...
34465,SD25-0419225,3,1,0,0,0,0,3,0
34466,SD25-0419607,1,0,0,0,0,0,1,0
34467,SD25-0423742,1,1,0,0,0,0,1,0
34468,SD25-0426418,2,1,0,0,0,0,1,0


## Payments data
<ol><li>Exclude collections for current month and making anniversary date of datetime data type</li>
<li>Split claims and monthly premiums</li>
<li>Aggregate claims data</li>
<li>Adding a paid flag column, paid?</li>
<li>Aggregating to policy_id and noting which of last n months premiums were paid.</li>
<li>Adding the duration to next anniversary</li>
<li>Aggregate to policy_id</li>
<li>Create payment rate column (no. paid/total payments due) and create no. of anniversaries column</li>
<li>Merge last months paid data and claims data to the aggrewgated payments data</li>
</ol>

In [30]:
payment_data

,policy_id,collections_id,sales_channel,collection_date,cover_start_date_original,cover_start_date,cancellation_effective_date,* policy start delay months,* policy duration months,anniversary_due,amount,collected_amount,collection_status,collection_sub_type,payment_method,payment_type,payment_provider,previous_collection_rate
0,1e3163bd-fdc3-318d-2a9f-6629f7e5daec,180d7f3f-0f72-4da0-964d-eaa545c1b45a,Agent,2024-08-27,2015-04-01,2015-04-01,NaT,0,124,2026-04-01 00:00:00,273.07,273.07,successful,Monthly,EFT,DebiCheck,Qlink,1.0
1,1e3163bd-fdc3-318d-2a9f-6629f7e5daec,268ae25c-2c77-4b83-a549-b5aced1fe352,Agent,2024-07-26,2015-04-01,2015-04-01,NaT,0,124,2026-04-01 00:00:00,273.07,273.07,successful,Monthly,EFT,DebiCheck,Qlink,1.0
2,1e3163bd-fdc3-318d-2a9f-6629f7e5daec,2ed02008-ba68-4bed-bc8e-fdef080c7bf2,Agent,2025-06-27,2015-04-01,2015-04-01,NaT,0,124,2026-04-01 00:00:00,294.92,294.92,successful,Monthly,EFT,DebiCheck,Qlink,1.0
3,1e3163bd-fdc3-318d-2a9f-6629f7e5daec,39d7a57b-a938-4afe-b9db-bc74167f436f,Agent,2024-10-25,2015-04-01,2015-04-01,NaT,0,124,2026-04-01 00:00:00,273.07,273.07,successful,Monthly,EFT,DebiCheck,Qlink,1.0
4,1e3163bd-fdc3-318d-2a9f-6629f7e5daec,4d8515c8-f6e0-4cdb-aad8-0c3baaa31333,Agent,2024-12-27,2015-04-01,2015-04-01,NaT,0,124,2026-04-01 00:00:00,273.07,273.07,successful,Monthly,EFT,DebiCheck,Qlink,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135596,f3941032-11e9-70a6-9453-682716baa954,dcc2f0e0-50b5-417c-8dcc-f64242b08051,Agent,2025-06-30,2025-06-01,2025-06-01,NaT,0,2,2026-06-01 00:00:00,351.43,351.43,successful,Monthly,DebiCheck,DebiCheck,Qlink Debicheck,1.0
135597,f3941032-11e9-70a6-9453-682716baa954,f2719079-c082-4c4d-ab1f-08a11e98cdfe,Agent,2025-07-31,2025-06-01,2025-06-01,NaT,0,2,2026-06-01 00:00:00,351.43,351.43,successful,Monthly,DebiCheck,DebiCheck,Qlink Debicheck,1.0
135598,f6270840-79cf-45dd-ba57-67f8ed6c7296,530d00d8-3ff2-4407-9d9a-0b2689d57061,Digital - Assisted,2025-06-26,2025-06-01,2025-06-01,NaT,0,2,2026-06-01 00:00:00,384.63,384.63,successful,Monthly,DebiCheck,DebiCheck,Qlink Debicheck,1.0
135599,f6270840-79cf-45dd-ba57-67f8ed6c7296,7e5deee2-b79c-4d1b-b9f9-2364fb37037a,Digital - Assisted,2025-05-26,2025-06-01,2025-06-01,NaT,0,2,2026-06-01 00:00:00,384.63,384.63,successful,Monthly,DebiCheck,DebiCheck,Qlink Debicheck,NaN


In [ ]:
#Exclude payments for current month
payment_data = payment_data[payment_data['collection_date']< pd.Timestamp('2025-08-01')]
#making anniversary date a datetime dtype
payment_data['anniversary_due'] = pd.to_datetime(payment_data['anniversary_due'], errors='coerce')

In [ ]:
claims_data = payment_data[payment_data['collection_sub_type']=='Claim']
payment_data = payment_data[payment_data['collection_sub_type']=='Monthly']

In [34]:
claims = DataPreparation(df=claims_data)
agg_dict = {'# claims': pd.NamedAgg(column='collections_id', aggfunc=pd.Series.nunique)}
claims.aggregation(groupby_cols=['policy_id'],cat_cols=[],agg_dict_not_cat_cols=agg_dict)
claims.agg_df

,policy_id,# claims
0,107c0513-5c34-9300-2ac3-6629ffb12bab,1
1,109f5129-16ca-89a7-1d26-65fb7dbce6b7,1
2,10b94d19-f6a2-a758-ecee-65fb935f5f54,2
3,112ecc3d-9ab4-4de8-8f73-65fb435e0f95,1
4,11408adb-e64a-93bd-71aa-65fb91571d7b,1
...,...,...
749,f383cdf7-9848-4573-da9d-65d8d67a6e7c,1
750,f409c8e6-f58b-14b7-a488-645e01853332,1
751,f62405ae-1eef-0e78-ce6b-65fb7e418d31,1
752,f75a0bad-4d18-cf62-be41-65fb7f1d2819,1


In [48]:
payment_data['paid?']= payment_data['collected_amount'].apply(lambda x: 'paid' if pd.notnull(x) else 'unpaid')

C:\Users\E1005279\AppData\Local\Temp\ipykernel_28560\2757173233.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  payment_data['paid?']= payment_data['collected_amount'].apply(lambda x: 'paid' if pd.notnull(x) else 'unpaid')


In [78]:

def last_n_payments(group):
    sortby='collection_date'
    n=4
    new_val = 'paid?'
    sorted_group = group.sort_values(sortby,ascending=False)
    top_n = sorted_group[new_val].head(n).tolist()
    top_n += ['N/A']*(n-len(top_n))
    new_cols=[]
    for m in range(0,n):
        new_cols.append(str(m+1)+' month ago payment')
    return pd.Series(top_n, index=new_cols)

last_month_payments_data = payment_data.groupby('policy_id', as_index=False).apply(last_n_payments)

C:\Users\E1005279\AppData\Local\Temp\ipykernel_28560\3109358826.py:13: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  last_month_payments_data = payment_data.groupby('policy_id', as_index=False).apply(last_n_payments)


In [79]:
last_month_payments_data.columns

Index(['policy_id', '1 month ago payment', '2 month ago payment',
       '3 month ago payment', '4 month ago payment'],
      dtype='object')

In [ ]:

payment_data['duration to anniversary (months)'] = payment_data.apply(
    lambda row: ((row['anniversary_due'] - row['cancellation_effective_date']) / np.timedelta64(1, 'D'))//30
    if pd.notnull(row['cancellation_effective_date'])
    else ((row['anniversary_due'] - pd.Timestamp('2025-08-01')) / np.timedelta64(1, 'D'))//30,
    axis=1
)


C:\Users\E1005279\AppData\Local\Temp\ipykernel_28560\1292687195.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  payment_data['duration to anniversary'] = payment_data.apply(


,policy_id,collections_id,sales_channel,collection_date,cover_start_date_original,cover_start_date,cancellation_effective_date,* policy start delay months,* policy duration months,anniversary_due,amount,collected_amount,collection_status,collection_sub_type,payment_method,payment_type,payment_provider,previous_collection_rate,paid?,duration to anniversary
0,1e3163bd-fdc3-318d-2a9f-6629f7e5daec,180d7f3f-0f72-4da0-964d-eaa545c1b45a,Agent,2024-08-27,2015-04-01,2015-04-01,NaT,0,124,2026-04-01,273.07,273.07,successful,Monthly,EFT,DebiCheck,Qlink,1.0,paid,8.0
1,1e3163bd-fdc3-318d-2a9f-6629f7e5daec,268ae25c-2c77-4b83-a549-b5aced1fe352,Agent,2024-07-26,2015-04-01,2015-04-01,NaT,0,124,2026-04-01,273.07,273.07,successful,Monthly,EFT,DebiCheck,Qlink,1.0,paid,8.0
2,1e3163bd-fdc3-318d-2a9f-6629f7e5daec,2ed02008-ba68-4bed-bc8e-fdef080c7bf2,Agent,2025-06-27,2015-04-01,2015-04-01,NaT,0,124,2026-04-01,294.92,294.92,successful,Monthly,EFT,DebiCheck,Qlink,1.0,paid,8.0
3,1e3163bd-fdc3-318d-2a9f-6629f7e5daec,39d7a57b-a938-4afe-b9db-bc74167f436f,Agent,2024-10-25,2015-04-01,2015-04-01,NaT,0,124,2026-04-01,273.07,273.07,successful,Monthly,EFT,DebiCheck,Qlink,1.0,paid,8.0
4,1e3163bd-fdc3-318d-2a9f-6629f7e5daec,4d8515c8-f6e0-4cdb-aad8-0c3baaa31333,Agent,2024-12-27,2015-04-01,2015-04-01,NaT,0,124,2026-04-01,273.07,273.07,successful,Monthly,EFT,DebiCheck,Qlink,1.0,paid,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135596,f3941032-11e9-70a6-9453-682716baa954,dcc2f0e0-50b5-417c-8dcc-f64242b08051,Agent,2025-06-30,2025-06-01,2025-06-01,NaT,0,2,2026-06-01,351.43,351.43,successful,Monthly,DebiCheck,DebiCheck,Qlink Debicheck,1.0,paid,10.0
135597,f3941032-11e9-70a6-9453-682716baa954,f2719079-c082-4c4d-ab1f-08a11e98cdfe,Agent,2025-07-31,2025-06-01,2025-06-01,NaT,0,2,2026-06-01,351.43,351.43,successful,Monthly,DebiCheck,DebiCheck,Qlink Debicheck,1.0,paid,10.0
135598,f6270840-79cf-45dd-ba57-67f8ed6c7296,530d00d8-3ff2-4407-9d9a-0b2689d57061,Digital - Assisted,2025-06-26,2025-06-01,2025-06-01,NaT,0,2,2026-06-01,384.63,384.63,successful,Monthly,DebiCheck,DebiCheck,Qlink Debicheck,1.0,paid,10.0
135599,f6270840-79cf-45dd-ba57-67f8ed6c7296,7e5deee2-b79c-4d1b-b9f9-2364fb37037a,Digital - Assisted,2025-05-26,2025-06-01,2025-06-01,NaT,0,2,2026-06-01,384.63,384.63,successful,Monthly,DebiCheck,DebiCheck,Qlink Debicheck,NaN,paid,10.0


In [68]:
payment_data.columns

Index(['policy_id', 'collections_id', 'sales_channel', 'collection_date',
       'cover_start_date_original', 'cover_start_date',
       'cancellation_effective_date', '* policy start delay months',
       '* policy duration months', 'anniversary_due', 'amount',
       'collected_amount', 'collection_status', 'collection_sub_type',
       'payment_method', 'payment_type', 'payment_provider',
       'previous_collection_rate', 'paid?', 'duration to anniversary',
       'policy duration (months)'],
      dtype='object')

In [71]:
payments = DataPreparation(df=payment_data)
agg_dict={'sales_channel': pd.NamedAgg(column='sales_channel', aggfunc='first'),
          '* policy start delay months': pd.NamedAgg(column='* policy start delay months', aggfunc='max'),
          '* policy duration months': pd.NamedAgg(column='* policy duration months', aggfunc='max'),
          'last premium amount due': pd.NamedAgg(column='amount', aggfunc='last'),
          'duration to anniversary': pd.NamedAgg(column='duration to anniversary', aggfunc='max')}
payments.aggregation(groupby_cols=['policy_id'],cat_cols=['paid?','payment_method'],agg_dict_not_cat_cols=agg_dict)
payments.agg_df

,policy_id,sales_channel,* policy start delay months,* policy duration months,last premium amount due,duration to anniversary,paid?_paid_sum,paid?_unpaid_sum,payment_method_DebiCheck_sum,payment_method_EFT_sum,payment_method_Pre Fund_sum
0,100003c9-2094-3107-e604-65d8e6b4ec03,Agent,0,45,624.06,3.0,17,0,6,11,0
1,10000cc8-8f08-5e2c-b6ad-661d48606710,Agent,0,14,249.27,10.0,15,0,12,3,0
2,100307d2-6857-7f50-833c-66b391c37879,Agent,0,11,288.91,1.0,11,4,11,4,0
3,10048ce0-253d-aeff-4f59-64e76756bddf,Agent,0,22,193.82,2.0,23,0,12,11,0
4,10061226-ed6a-b733-47f0-65fb970086cd,Agent,0,48,388.10,0.0,16,0,0,16,0
...,...,...,...,...,...,...,...,...,...,...,...
63976,ffaa032e-2a01-cf52-cc1a-65fb3147c393,Agent,0,73,602.75,-2.0,15,4,0,19,0
63977,ffb7087c-78f7-a485-5b7f-65d8d25da2c7,Agent,0,62,349.90,10.0,17,2,0,19,0
63978,ffd20919-303c-e88e-fb1f-65fba1578790,Agent,0,34,200.68,2.0,15,1,0,16,0
63979,ffe1059c-beea-354d-27fa-6629fe94a5cf,Agent,0,80,883.94,4.0,15,1,0,16,0


In [72]:
payments.agg_df['# anniversaries']=payments.agg_df['* policy duration months']//12
payments.agg_df['payment rate']=payments.agg_df['paid?_paid_sum']/(payments.agg_df['paid?_paid_sum']+payments.agg_df['paid?_unpaid_sum'])

In [73]:
payments.agg_df

,policy_id,sales_channel,* policy start delay months,* policy duration months,last premium amount due,duration to anniversary,paid?_paid_sum,paid?_unpaid_sum,payment_method_DebiCheck_sum,payment_method_EFT_sum,payment_method_Pre Fund_sum,# anniversaries,payment rate
0,100003c9-2094-3107-e604-65d8e6b4ec03,Agent,0,45,624.06,3.0,17,0,6,11,0,3,1.000000
1,10000cc8-8f08-5e2c-b6ad-661d48606710,Agent,0,14,249.27,10.0,15,0,12,3,0,1,1.000000
2,100307d2-6857-7f50-833c-66b391c37879,Agent,0,11,288.91,1.0,11,4,11,4,0,0,0.733333
3,10048ce0-253d-aeff-4f59-64e76756bddf,Agent,0,22,193.82,2.0,23,0,12,11,0,1,1.000000
4,10061226-ed6a-b733-47f0-65fb970086cd,Agent,0,48,388.10,0.0,16,0,0,16,0,4,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63976,ffaa032e-2a01-cf52-cc1a-65fb3147c393,Agent,0,73,602.75,-2.0,15,4,0,19,0,6,0.789474
63977,ffb7087c-78f7-a485-5b7f-65d8d25da2c7,Agent,0,62,349.90,10.0,17,2,0,19,0,5,0.894737
63978,ffd20919-303c-e88e-fb1f-65fba1578790,Agent,0,34,200.68,2.0,15,1,0,16,0,2,0.937500
63979,ffe1059c-beea-354d-27fa-6629fe94a5cf,Agent,0,80,883.94,4.0,15,1,0,16,0,6,0.937500


In [83]:
payments_data = multi_data_ops(file_list=[],df1=payments.agg_df,df2=last_month_payments_data,df3=claims.agg_df)
pay_last = payments_data.merging(on='policy_id',df_left=payments_data.df1, df_right=payments_data.df2, how='left',col_left=['policy_id', 'sales_channel', '* policy start delay months',
        '* policy duration months', 'last premium amount due',
        'duration to anniversary','payment_method_DebiCheck_sum', 'payment_method_EFT_sum',
        'payment_method_Pre Fund_sum', '# anniversaries', 'payment rate'])
payments_data.merged_df = payments_data.merging(on='policy_id',df_left=pay_last, df_right=payments_data.df3, how='left')
payments_data.merged_df['# claims'] = payments_data.merged_df['# claims'].apply(lambda x: x if pd.notnull(x) else 0)
payments_data.merged_df

,policy_id,sales_channel,* policy start delay months,* policy duration months,last premium amount due,duration to anniversary,payment_method_DebiCheck_sum,payment_method_EFT_sum,payment_method_Pre Fund_sum,# anniversaries,payment rate,1 month ago payment,2 month ago payment,3 month ago payment,4 month ago payment,# claims
0,100003c9-2094-3107-e604-65d8e6b4ec03,Agent,0,45,624.06,3.0,6,11,0,3,1.000000,paid,paid,paid,paid,0.0
1,10000cc8-8f08-5e2c-b6ad-661d48606710,Agent,0,14,249.27,10.0,12,3,0,1,1.000000,paid,paid,paid,paid,0.0
2,100307d2-6857-7f50-833c-66b391c37879,Agent,0,11,288.91,1.0,11,4,0,0,0.733333,unpaid,unpaid,paid,paid,0.0
3,10048ce0-253d-aeff-4f59-64e76756bddf,Agent,0,22,193.82,2.0,12,11,0,1,1.000000,paid,paid,paid,paid,0.0
4,10061226-ed6a-b733-47f0-65fb970086cd,Agent,0,48,388.10,0.0,0,16,0,4,1.000000,paid,paid,paid,paid,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63976,ffaa032e-2a01-cf52-cc1a-65fb3147c393,Agent,0,73,602.75,-2.0,0,19,0,6,0.789474,unpaid,unpaid,paid,paid,0.0
63977,ffb7087c-78f7-a485-5b7f-65d8d25da2c7,Agent,0,62,349.90,10.0,0,19,0,5,0.894737,paid,paid,unpaid,paid,0.0
63978,ffd20919-303c-e88e-fb1f-65fba1578790,Agent,0,34,200.68,2.0,0,16,0,2,0.937500,paid,paid,paid,paid,0.0
63979,ffe1059c-beea-354d-27fa-6629fe94a5cf,Agent,0,80,883.94,4.0,0,16,0,6,0.937500,paid,unpaid,paid,paid,0.0


## Lapse data


In [95]:
lapse_data.columns

Index(['policy_name', 'age', 'cover_amount_full', 'policy_status',
       'current_policy_status', 'education', 'eml', 'pml',
       'expected_lapse_rate', 'exposure_status', 'gender', 'hiv_group',
       'individual_income', 'lapse_type', 'num_unpaid_months',
       'occupation_class', 'partner_income', 'policy_sale_date',
       'smoker_status'],
      dtype='object')

In [ ]:
lapse = DataPreparation(df=lapse_data)
agg_dict={'cover_amount_full': pd.NamedAgg(column='cover_amount_full', aggfunc='mean'),
          'current_policy_status': pd.NamedAgg(column='current_policy_status', aggfunc='last'),
          'education': pd.NamedAgg(column='education', aggfunc='last'),
          'eml': pd.NamedAgg(column='eml', aggfunc='mean'),
          'pml': pd.NamedAgg(column='pml', aggfunc='mean'),
          'expected_lapse_rate': pd.NamedAgg(column='expected_lapse_rate', aggfunc='mean'),
          'gender': pd.NamedAgg(column= 'gender', aggfunc='last'),
          'hiv_group': pd.NamedAgg(column='hiv_group', aggfunc='last'),
          'current individual_income': pd.NamedAgg(column='individual_income', aggfunc='last'),
          'orginal individual_income': pd.NamedAgg(column='individual_income', aggfunc='first'),
          'lapse_type': pd.NamedAgg(column='lapse_type', aggfunc='mode'),
          'occupation_class': pd.NamedAgg(column='occupation_class', aggfunc='last'),
          'partner_income': pd.NamedAgg(column='partner_income', aggfunc='last'),
          'smoker_status': pd.NamedAgg(column='smoker_status', aggfunc='last')}
lapse.aggregation(groupby_cols=['policy_name'],cat_cols=[],agg_dict_not_cat_cols=agg_dict)
lapse.agg_df

,policy_name,age,cover_amount_full,policy_status,current_policy_status,education,eml,pml,expected_lapse_rate,exposure_status,gender,hiv_group,individual_income,lapse_type,num_unpaid_months,occupation_class,partner_income,policy_sale_date,smoker_status
0,MWL000004693,48.0000,295491.00,Cancelled,Cancelled,No Matric,0.00,0.0,0.009202,On-risk : Paid Cover,Female,amber,14000.0,NaN,NaN,NaN,0,2015-03-03,Non Smoker
1,MWL000004693,48.0833,295491.00,Cancelled,Cancelled,No Matric,0.00,0.0,0.009202,On-risk : Paid Cover,Female,amber,14000.0,NaN,NaN,NaN,0,2015-03-03,Non Smoker
2,MWL000004693,48.1667,310265.55,Cancelled,Cancelled,No Matric,0.00,0.0,0.009202,On-risk : Paid Cover,Female,amber,14000.0,NaN,NaN,NaN,0,2015-03-03,Non Smoker
3,MWL000004693,48.2500,310265.55,Cancelled,Cancelled,No Matric,0.00,0.0,0.009202,On-risk : Free Cover,Female,amber,14000.0,NaN,NaN,NaN,0,2015-03-03,Non Smoker
4,MWL000004693,48.3333,310265.55,Cancelled,Cancelled,No Matric,0.00,0.0,0.009202,On-risk : Paid Cover,Female,amber,14000.0,Client Selected,NaN,NaN,0,2015-03-03,Non Smoker
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
917341,SD25-0426418,8.0833,50000.00,Cancellation Requested,Cancellation Requested,No Matric,0.00,0.0,NaN,On-risk : Paid Cover,Female,red,16000.0,NaN,NaN,A,0,2025-07-29,Non Smoker
917342,SD25-0426418,32.0000,1000000.00,Cancellation Requested,Cancellation Requested,No Matric,0.35,0.0,NaN,On-risk : Paid Cover,Female,red,16000.0,NaN,NaN,A,0,2025-07-29,Non Smoker
917343,SD25-0426488,45.2500,500000.00,Active,Active,No Matric,-0.05,0.0,NaN,On-risk : Paid Cover,Female,red,34000.0,NaN,NaN,C,0,2025-07-29,Non Smoker
917344,ML25-0427422,30.2500,3500000.00,Active,Active,Postgraduate Degree,-0.05,0.0,0.108399,On-risk : Paid Cover,Female,red,30000.0,NaN,NaN,A,0,2025-07-30,Non Smoker


In [97]:
lapse.agg_df['policy_name'].nunique()

65364

# Merging all the data

In [6]:
data = multi_data_ops(file_list=[],df1=calls_data, df2=sales_data,df3=policy_data,df4=payment_data)

In [8]:
data.col_intersect

{'df1 | df2': set(),
 'df1 | df3': {'policy_name'},
 'df1 | df4': set(),
 'df2 | df3': {'campaign_name', 'policy_id'},
 'df2 | df4': {'policy_id', 'sales_channel'},
 'df3 | df4': {'anniversary_due',
  'cancellation_effective_date',
  'cover_start_date',
  'cover_start_date_original',
  'payment_type',
  'policy_id'}}